In [1]:
import pandas as pd

In [6]:
long = pd.read_csv('Turtle_BNF.csv')
short = pd.read_csv('Turtle-Shorting.csv')

In [7]:
combined_df = pd.concat([long, short])


In [8]:
combined_df = combined_df.sort_values(by='Entry_Date')
combined_df.tail()

,Unnamed: 0,index,Trade No.,Entry_Date,Exit_Date,Trade_Type,Entry_Price,Previous MA Value,Initial SL,Exit_Price,Exit Remark,PnL,Slippage,PnL w cs,Qty,Profit_Loss INR,ROI%,Trade Year
667,NaN,667,667,2025-01-21 10:45:00+05:30,2025-01-21 11:45:00+05:30,short,48745.10,48865.636538,49044.405769,49044.405769,Initial SL Hit,-299.305769,9.778951,-309.084720,83.526623,-25816.802713,-2.581680,2025
668,NaN,668,668,2025-01-21 12:15:00+05:30,2025-01-22 14:15:00+05:30,short,48681.65,48908.336538,49096.876923,48567.700000,Trailing SL Hit,113.950000,9.724935,104.225065,60.208042,6275.187085,0.627519,2025
645,645.0,645,645,2025-01-22 14:45:00+05:30,2025-01-23 09:15:00+05:30,Long,48762.50,48523.733333,48324.279167,48589.150000,Trailing SL Hit,-173.350000,9.735165,-183.085165,14.262216,-2611.200094,-1.044480,2025
646,646.0,646,646,2025-01-23 09:45:00+05:30,2025-01-23 11:15:00+05:30,Long,48788.85,48545.666667,48345.600000,48615.200000,Trailing SL Hit,-173.650000,9.740405,-183.390405,14.100395,-2585.877115,-1.034351,2025
647,647.0,647,647,2025-01-24 10:45:00+05:30,NaN,Long,48790.35,48690.120833,48537.333333,NaN,NaN,NaN,NaN,NaN,24.701930,NaN,NaN,2025


In [9]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [10]:
stats = generate_stats(combined_df, 'Combined L+S')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,0.0,0,NaN,NaN,NaN,NaN,NaN,Combined L+S
2018,150.97051,174,32.183908,6.070256,-1.601388,-30.699122,4.917747,Combined L+S
2019,132.208385,185,34.594595,5.187789,-1.651323,-24.325542,5.434961,Combined L+S
2020,199.074448,197,32.48731,6.154629,-1.464826,-17.115203,11.63144,Combined L+S
2021,158.824481,205,31.707317,5.761723,-1.540625,-19.858934,7.997634,Combined L+S
2022,76.367029,185,28.108108,5.529922,-1.587886,-40.652216,1.878545,Combined L+S
2023,118.164757,186,27.419355,6.077268,-1.420562,-27.150336,4.352239,Combined L+S
2024,109.440135,172,34.883721,5.061933,-1.734606,-23.84682,4.589297,Combined L+S
Overall,945.049743,1304,31.66287,5.639644,-1.565457,-40.652216,23.247189,Combined L+S
